# __Segmentacion de Comercio Online__

## Importar librerias

In [1]:
# librerias para el dataframe y la visualizacion
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

## Cargar los datos

In [3]:
retail_df = pd.read_csv('./OnlineRetail.csv', sep=',', encoding='ISO-8859-1', header=0)
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


## Acerca de los datos

Este es un conjunto de datos transaccionales que contiene todas las transacciones realizadas entre el 01/12/2010 y el 09/12/2011 para una tienda online sin establecimientos físicos con sede en el Reino Unido. La empresa vende principalmente regalos únicos para cualquier ocasión. Muchos de sus clientes son mayoristas.

Las variables que contienen los datos son:

- _InvoiceNo_: Numero de factura. Es un numero entero de 6 digitos asignado de forma unica a cada transaccion. si este codigo empieza con la letra "c", indica una cancelacion.

- _StockCode_: Codigo de stock del producto. Es un numero de 5 digitos asignado de forma unica a cada producto.

- _Description_: Nombre del producto.

- _Quantity_: Cantidades de cada producto por transaccion.

- _InvoiceDate_: Fecha de factura. Marca el dia y la hora en que se genero cada transaccion.

- _UnitPrice_: Precio del producto por unidad.

- _CustomerID_: Numero integral de 5 digitos asignado de forma unica a cada cliente.

- _Country_: Nombre del pais donde reside cada cliente.